## Начало работы

**Подключаю нужные модули:**

In [ ]:
import numpy as np  # массивы, матрицы, математика
import pandas as pd  # данные, чтение и обработка
import scipy.stats as sps  # статистика
import matplotlib.pyplot as plt  # графики
import seaborn as sns  # графики
import os  # работа с файлами

**Создаю датафрейм:**

In [ ]:
# Создаю пустой список для хранения дф
dfs = []

# Указываю путь
file_path = "D:\\Datasets\\Clash_of_Clans\\coc_clans_dataset.csv"

# Читаю данные из CSV файла и сохраняю их в дф
df = pd.read_csv(file_path)

# Добавляю дф в список
dfs.append(df)

**Ознакамливаюсь с данными:**

In [ ]:
df.shape

In [ ]:
pd.set_option('display.max_columns', None)
df.head(10)

In [ ]:
df.info()

**Создаю копию df:**<br>
В копии data буду проводить изменения, а оригинальный датафрейм df останется для просмотра изначальных данных.

In [ ]:
data = df.copy()

# 2. Очистка данных и работа с пустыми значениями

**Поиск и удаление дубликатов:**

In [ ]:
data[data.duplicated()]

Дубликатов в данных нет, но если бы они были, то нужно было бы выполнить удаление:

In [ ]:
data.drop_duplicates()
data.shape

**Выявление и работа с пустыми значениями:**

In [ ]:
missing_values = df.isnull().sum()
print(missing_values)

В датасете отсутствуют данные только в 3 столбцах, имя клана, описание клана и регион. Однако, эти данные не являются критическими для моего исследования и не влияют на возможность идентификации каждого клана.<br>

**Посмотрю подробнее отсутствующие значения на каждом из уровней кланов:**

In [ ]:
# Пропуски в названии кланов
missing_name = df[df['clan_name'].isnull()]

# по уровням
missing_name_count_by_level = missing_name.groupby('clan_level').size()

missing_name_count_by_level

In [ ]:
# Пропуски в локации кланов
missing_location = df[df['clan_location'].isnull()]

# по уровням
missing_location_count_by_level = missing_location.groupby('clan_level').size()

missing_location_count_by_level

In [ ]:
# Пропуски в описании кланов
missing_description = df[df['clan_description'].isnull()]

# по уровням
missing_description_count_by_level = missing_description.groupby('clan_level').size()

missing_description_count_by_level

In [ ]:
# Фильтрую строки, в которых нет одновременно и названия, и описания, и локации клана
missing_all_count = df[df[['clan_name', 'clan_location', 'clan_description']].isnull().all(axis=1)]
missing_all_count

Строк где нет одновременно названия, описания и локации только 26 шт. Мне для моего исследования это никак не мешает. <br>
**Заменю все отстуствующие значения этих 3 столбцов на что-то понятное:**

In [ ]:
data['clan_name'] = data['clan_name'].fillna('Unknown')

In [ ]:
data['clan_description'] = data['clan_description'].fillna('Unknown')

In [ ]:
data['clan_location'] = data['clan_location'].fillna('Unknown')

In [ ]:
missing_values = data.isnull().sum()
missing_values

# Поиск и обработка аномалий (выбросов)

**Посмотрю какое количество кланов на каждом уровне**

In [ ]:
clan_level_counts = data.groupby('clan_level').size()
clan_level_counts

Большая часть кланов занимают 1 уровень, что нормально, так как туда попадают все новые кланы. В целом видно, что количество кланов по мере увеличения уровня пропорционально уменьшается. Однако стоит проверить, точно ли нет ошибок.

**Посмотрю соотношение побед к поражениям на диаграмме рассеяния (scatterplot):** <br>
Оно примерно должно быть пропорциональным.

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(df['war_losses'], df['war_wins'], color='blue', alpha=0.5)
plt.title('Соотношение побед к поражениям')
plt.xlabel('Число поражений')
plt.ylabel('Число побед')
plt.grid(True)
plt.show()

По распределению видно, что основная масса кланов имеет пропорциональное соотношение, однако выделяется полоса кланов не имеющих ни одного поражения,но с большим количеством побед. Также имеется часть кланов с обратной ситуацией. 

**Проверю war_win_streak (серия побед в войнах)** <br>
Посмотрю, есть ли строки, где количество побед равно или больше 2, серия побед равна 0, и количество поражений равно 0. В данной игре серия побед засчитывается если есть больше 1 победы подряд.

In [ ]:
war_stat_strange = df[(df['war_wins'] >= 2) & (df['war_win_streak'] == 0) & (df['war_losses'] == 0)]

#отобразить все столбцы
pd.set_option('display.max_columns', None)
war_stat_strange

184359 строки имеют аномальное количество побед по отношению к 0 сериям побед и проигрышам.

**Визуализирую эту ситуацию по уровням:**

In [ ]:
war_stat_strange_counts = war_stat_strange.groupby('clan_level').size()
war_stat_strange_counts

In [ ]:
war_stat_strange_counts.plot(kind='bar', figsize=(12, 4), color='red')
plt.title('Строки, где количество побед >= 2, а поражений и серии побед 0')
plt.xlabel('Уровень клана')
plt.ylabel('Количество строк')
plt.xticks(rotation = 25)
plt.grid(axis='y')
plt.show()

**Вынесу "странные строки" в отдельный датафрейм, а из data удалю:**

In [ ]:
# Выделяю выборку из data
war_stat_strange = data[(data['war_wins'] >= 2) & (data['war_win_streak'] == 0) & (data['war_losses'] == 0)].copy()

# Удаляю такие строки
data.drop(war_stat_strange.index, inplace=True)

Посмотрим, что изменилось:

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(data['war_losses'], data['war_wins'], color='blue', alpha=0.5)
plt.title('Соотношение побед к поражениям')
plt.xlabel('Число поражений')
plt.ylabel('Число побед')
plt.grid(True)
plt.show()

Подозрительный столбец с 0 проигрышами все еще остался, хоть и видно, что часть кланов пропала.<br> 
**Посмотрю в более крупном масштабе:**

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(data['war_losses'], data['war_wins'], color='blue', alpha=0.5)
plt.title('Соотношение побед к поражениям')
plt.xlabel('Число поражений')
plt.ylabel('Число побед')

plt.xlim(-1, 5)
plt.ylim(0, 70)

plt.grid(True)
plt.show()

Для своего исследования я уберу в отдельный датафрейм строки, которые имеют более 30 побед при 0 поражениях.

In [ ]:
zero_losses_from_30 = data[(data['war_wins'] >= 30) & (data['war_losses'] == 0)].copy()
zero_losses_from_30
# таких 44705 строк

In [ ]:
# Удаляю эти эти строки из data
data.drop(zero_losses_from_30.index, inplace=True)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(data['war_losses'], data['war_wins'], color='blue', alpha=0.5)
plt.title('Соотношение побед к поражениям')
plt.xlabel('Число поражений')
plt.ylabel('Число побед')
plt.grid(True)
plt.show()

**Я хочу проверить на аномалии самые выделяющиеся 5 областей:**

**ЧАСТЬ 1:**

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

# Выборка данных в выбранном диапазоне
selected_data = data[(data['war_losses'] >= 0) & (data['war_losses'] <= 200) & 
                     (data['war_wins'] >= 350) & (data['war_wins'] <= 800)]

X = selected_data[['war_losses', 'war_wins']].values

# LOF
lof = LocalOutlierFactor(n_neighbors=100, contamination=0.06)  # Настройте параметры по вашему усмотрению
lof_labels = lof.fit_predict(X)

# Определение аномальных строк
anomalous_data = selected_data[lof_labels == -1]
normal_data = selected_data[lof_labels != -1]

# Визуализация
plt.figure(figsize=(10, 6))
plt.scatter(normal_data['war_losses'], normal_data['war_wins'], color='blue', alpha=0.5, label='Нормальные')
plt.scatter(anomalous_data['war_losses'], anomalous_data['war_wins'], color='red', alpha=0.5, label='Аномалии')
plt.title('LOF')
plt.xlabel('Число поражений')
plt.ylabel('Число побед')
plt.legend()
plt.grid(True)
plt.show()

num_anomalous = anomalous_data.shape[0]
print(f'Количество аномальных строк: {num_anomalous}')

In [ ]:
# Удаление аномальных строк из основного DataFrame
anomalous_indices = anomalous_data.index
data = data.drop(anomalous_indices)

# Сброс индексов для полученного DataFrame без аномалий
data.reset_index(drop=True, inplace=True)

**ЧАСТЬ 2:**

In [ ]:

selected_data = data[(data['war_losses'] >= 0) & (data['war_losses'] <= 400) & 
                     (data['war_wins'] >= 500) & (data['war_wins'] <= 1200)]

X = selected_data[['war_losses', 'war_wins']].values

lof = LocalOutlierFactor(n_neighbors=100, contamination=0.05)  # Настройте параметры по вашему усмотрению
lof_labels = lof.fit_predict(X)

anomalous_data = selected_data[lof_labels == -1]
normal_data = selected_data[lof_labels != -1]

plt.figure(figsize=(10, 6))
plt.scatter(normal_data['war_losses'], normal_data['war_wins'], color='blue', alpha=0.5, label='Нормальные')
plt.scatter(anomalous_data['war_losses'], anomalous_data['war_wins'], color='red', alpha=0.5, label='Аномалии')
plt.title('LOF')
plt.xlabel('Число поражений')
plt.ylabel('Число побед')
plt.legend()
plt.grid(True)
plt.show()

num_anomalous = anomalous_data.shape[0]
print(f'Количество аномальных строк: {num_anomalous}')

In [ ]:
anomalous_indices = anomalous_data.index
data = data.drop(anomalous_indices)

data.reset_index(drop=True, inplace=True)

**ЧАСТЬ 3:**

In [ ]:

selected_data = data[(data['war_losses'] >= 400) & (data['war_losses'] <= 900) & 
                     (data['war_wins'] >= 600) & (data['war_wins'] <= 1000)]

X = selected_data[['war_losses', 'war_wins']].values

lof = LocalOutlierFactor(n_neighbors=100, contamination=0.07)  # Настройте параметры по вашему усмотрению
lof_labels = lof.fit_predict(X)

anomalous_data = selected_data[lof_labels == -1]
normal_data = selected_data[lof_labels != -1]

plt.figure(figsize=(10, 6))
plt.scatter(normal_data['war_losses'], normal_data['war_wins'], color='blue', alpha=0.5, label='Нормальные')
plt.scatter(anomalous_data['war_losses'], anomalous_data['war_wins'], color='red', alpha=0.5, label='Аномалии')
plt.title('LOF')
plt.xlabel('Число поражений')
plt.ylabel('Число побед')
plt.legend()
plt.grid(True)
plt.show()

num_anomalous = anomalous_data.shape[0]
print(f'Количество аномальных строк: {num_anomalous}')

In [ ]:
anomalous_indices = anomalous_data.index
data = data.drop(anomalous_indices)

data.reset_index(drop=True, inplace=True)

**ЧАСТЬ 4:**

In [ ]:
selected_data = data[(data['war_losses'] >= 400) & (data['war_losses'] <= 1300) & 
                     (data['war_wins'] >= 0) & (data['war_wins'] <= 600)]

X = selected_data[['war_losses', 'war_wins']].values

lof = LocalOutlierFactor(n_neighbors=300, contamination=0.03)  # Настройте параметры по вашему усмотрению
lof_labels = lof.fit_predict(X)

anomalous_data = selected_data[lof_labels == -1]
normal_data = selected_data[lof_labels != -1]

plt.figure(figsize=(10, 6))
plt.scatter(normal_data['war_losses'], normal_data['war_wins'], color='blue', alpha=0.5, label='Нормальные')
plt.scatter(anomalous_data['war_losses'], anomalous_data['war_wins'], color='red', alpha=0.5, label='Аномалии')
plt.title('LOF')
plt.xlabel('Число поражений')
plt.ylabel('Число побед')
plt.legend()
plt.grid(True)
plt.show()

num_anomalous = anomalous_data.shape[0]
print(f'Количество аномальных строк: {num_anomalous}')

In [ ]:
anomalous_indices = anomalous_data.index
data = data.drop(anomalous_indices)

data.reset_index(drop=True, inplace=True)

**Смотрю, как сейчас выглядят данные:**

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(data['war_losses'], data['war_wins'], color='blue', alpha=0.5)
plt.title('Соотношение побед к поражениям')
plt.xlabel('Число поражений')
plt.ylabel('Число побед')
plt.grid(True)
plt.show()

In [ ]:
data.shape
# Было 3559743

# Влияние уровня ратуши клана и его столицы на результаты в войнах